**Get the data for the model training**

In this part of the code we will import the RNA-seq and CITE-seq data and start exploring potential models. First we would get the data in and convert it into the standard data format.

In [2]:

# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc



### Import the data and convert

In [3]:

# protein expression data
pd_prot_exp = pd.read_csv('/home/ec2-user/modal_form/CrossModalFormer_CITE_Seq/Data/documentation/Protein_expression.csv')


In [5]:

pd_prot_exp.head(5)


,Protein,CELL_1,CELL_2,CELL_3,CELL_4,CELL_5,CELL_6,CELL_7,CELL_8,CELL_9,...,CELL_218322,CELL_218323,CELL_218324,CELL_218325,CELL_218326,CELL_218327,CELL_218328,CELL_218329,CELL_218330,CELL_218331
0,CD117 protein,0.000000,0.000000,0.405465,0.000000,0.000000,0.000000,0.000000,0.000000,1.609438,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.693147
1,CD119 protein,1.386294,1.504077,1.871802,1.945910,2.079442,1.098612,1.791759,1.386294,1.609438,...,2.079442,1.791759,1.94591,1.252763,1.94591,0.693147,1.609438,1.386294,1.098612,1.098612
2,CD140a protein,0.693147,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,CD140b protein,0.693147,0.000000,0.000000,0.000000,0.693147,0.693147,0.000000,0.693147,1.098612,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,CD172a protein,0.000000,1.098612,0.000000,1.098612,0.693147,0.693147,0.693147,1.098612,0.693147,...,0.000000,0.693147,0.00000,0.693147,0.00000,0.000000,0.693147,0.000000,0.000000,0.000000


In [6]:

# RNA expression data
adata_1 = pd.read_csv('/home/ec2-user/modal_form/CrossModalFormer_CITE_Seq/Data/expression/RNA_expression_subset1a.csv')

In [ ]:

# need to convert csv files to adata.h5ad files


### Try simple linear regression model

In [ ]:

# find common columns
cell_idx = np.intersect1d(adata_1.columns, pd_prot_exp.columns)

: 

In [ ]:

# RNA and protein
X_rna = adata_1[cell_idx]
Y_protein = pd_prot_ext[cell_idx]



In [ ]:

Y_protein
